# Transaction module
進行交易

In [10]:
import numpy as np 
import pandas as pd
from helper_func import init_account, init_rate, get_one_stock
from profit_loss import profit_loss_analysis

## Initial Transaction info

In [ ]:
# def init_account(shares=0.0, cost=0.0, buy_times=0.0, earn=0.0,sell_times=0.0, capital=300000,per_trans=1000):
#     account = {
#         'shares': shares,     # 有幾股
#         'cost': cost,
#         'buy_times': buy_times,
#         'earn': earn,
#         'sell_times': sell_times,
#         'capital': capital,  # 資金
#         'per_trans': per_trans,   # 一次買一張
#         'init_capital': capital,  # 紀錄初始資金
#     }
#     return account

In [ ]:
def init_transc(strategy = 1):
    info = None
    if strategy == 1:
        info = {
            'close': 0,
            'pre_close': 0,
            'mean': 0,
            'stop_profit': 0,
            'stop_loss': 0,
            'have_day' : 0
        }
    return info

## Transaction Buy & Sell

In [ ]:
def buy(investor, price, rate):
    b_price = investor['per_trans'] * (price + rate['slippage'])
    b_price += b_price * rate['handling_fee']
    investor['cost'] += b_price
    investor['buy_times'] += 1
    investor['shares'] += investor['per_trans']
    investor['capital'] -= b_price
    return investor, b_price

In [ ]:
def sell(investor, price, rate):
    b_price = investor['shares'] * (price + rate['slippage'])
    b_price *= (1 - rate['handling_fee'])
    b_price *= (1 - rate['trans_tax'])
    investor['earn'] += b_price
    investor['sell_times'] += 1
    investor['shares'] = 0
    investor['capital'] += b_price
    return investor, b_price

## Stop Loss & Stop Profit

In [ ]:
# 更新停損停利點
def update_info(info, company, day, investor):
    
    info['close'] = company['close'][day]
    info['mean'] = company['middle'][day]
    
    if investor['shares'] == 0:
        info['have_day'] = 0
    else:
        info['have_day'] += 1
    return info

In [ ]:
def update_stop_point(info, raise_rate, drop_rate):
    info['stop_profit'] = info['close'] * (1 + raise_rate)
    info['stop_loss'] = info['close'] * (1 - drop_rate)
    return info

## Trade Strategy

In [ ]:
def simple_strategy(info, expiry_period = 60):
    
    if info['close'] > info['mean'] and info['pre_close'] < info['mean']:
        sign = "cross_up"
#     elif info['close'] < info['mean'] and info['pre_close'] > info['mean']:
#         sign = "cross_dwn"
#         sign = "None"
    elif info['close'] >= info['stop_profit']:
        sign = "stop_profit"
    elif info['close'] <= info['stop_loss']:
        sign = "stop_loss"
#     elif info['have_day'] >= expiry_period:
#         sign = "expiry_date"
    else:
        sign = "None"

    return sign, info['close']

In [ ]:
# Return sign, price
def trade(info, strategy = 1):
    if strategy == 1:
        return simple_strategy(info)

## Transaction

In [3]:
# return company dataframe, transaction record, investor final
def transaction(company, strategy=1, drop_rate=0.03, raise_rate=0.02, print_earn=False):
    # Investor capital
    investor = init_account()
    SIG = np.zeros(len(company))
    rate = init_rate()

    record = []
    
    # Initial trading info
    info = init_transc()
    
    for day in range(0, len(company)):
        
        
        info = update_info(info, company, day, investor)
        
        BUY = ["cross_up"]
        SELL = ["cross_dwn", "stop_loss", "stop_profit", "expiry_date"]
        
        sign, price = trade(info)

        if sign in BUY and investor['shares'] == 0:
            info = update_stop_point(info, raise_rate, drop_rate)
            investor, actuall_price = buy(investor, price, rate)
            record.append([company.index[day], 'BUY', sign, investor['capital'],
                           investor['shares'], price, actuall_price, day])
            SIG[day] = 1
        elif sign in SELL and investor['shares'] != 0:
            investor, actuall_price = sell(investor, price, rate)
            record.append([company.index[day], 'SELL', sign, investor['capital'],
                           investor['shares'], price, actuall_price, day])
            SIG[day] = 2
        
        info['pre_close'] = info['close']
        
    # 清空
    if investor['shares'] != 0:
        investor, actuall_price = sell(investor, price, rate)
        record.append([company.index[day], 'SELL', 'clean', investor['capital'],
                           investor['shares'], info['close'], actuall_price, day])
        SIG[day] = 2
    total_earn = investor['earn'] - investor['cost']
    if print_earn:
        print(f"Total Earn: {total_earn}")
    record = pd.DataFrame(record, columns=['日期', '動作', '指標', '總資金', '持有股', '交易每股金額', '實際買賣金額', 'Index'])
    return record, investor, SIG, total_earn

## Get Transaction DF

In [ ]:
# 拿到交易分析 dataFrame 
# ticker: 股票代碼
def get_transaction_DF(ticker , basic_ratio=None):

    # 拿到股票資訊
    company = get_one_stock(ticker)

    # 停利停損
    if basic_ratio is None:
        basic_ratio = np.arange(2,5,1) # 停利為停損的幾倍
    drop_ratios = np.arange(0.01, 0.1, 0.01) # 停損

    # 紀錄
    records = []

    # r 為倍率: 停損的r倍
    for r in basic_ratio:
        for d in drop_ratios:
            _raise_rate = d*r
            # recored: 交易紀錄 investor:交易者資訊 total_earn: 賺多少 
            record, investor, _, total_earn = transaction(company, drop_rate=d, raise_rate=_raise_rate) 
            # WL_rate: 賺賠比 pw: 勝率   
            WL_rate, pw, _ = profit_loss_analysis(record)
            hpr = round(total_earn / investor['init_capital'] * 100 ,2) # 報酬率:%
            # 紀錄
            records.append( [_raise_rate , d , investor['buy_times'] , investor['sell_times'] , WL_rate , pw , hpr ]   )

    df = pd.DataFrame(records ,columns=['停利點','停損點','進場次數','出場次數','W/L','Pw','投資報酬率(%)'])
    return df